# Histogram Objects

In [1]:
import qcware.config
# this line is for internal tracking; it is not necessary for use!
qcware.config.set_environment_source_file('measurement.ipynb')


import quasar
from qcware.circuits.quasar_backend import QuasarBackend
# as always, when using Forge we must set our API key
import qcware

## Introduction

A fundamental operation encountered in qubit hardware is the process of measurement. Generally, one prepares a quantum state by running a quantum circuit and then projectively measures each qubit to determine if that qubit ends up in the $|0\rangle$ or $|1\rangle$ state. The collective measurement result over all qubits is a binary string, e.g., $|0011\rangle$, which we refer to as a `Ket`. This procedure is repeated `nmeasurement` times, with different `Ket`s appearing probabilistically according to the square of the amplitude of each `Ket` in the quantum state. The fundamental output from this overall process is a histogram of `Ket` strings observed together with the the probability that each one was observed and the total number of measurements. We refer to this overall output as a `ProbabilityHistogram` object. An alternative output in terms of histogram of `Ket` strings observed together with the the integral number of times that each one was observed is provided as the `CountHistogram` object - utility functions exist to quickly convert back and forth between `ProbabilityHistogram` and `CountHistogram`.

**Naming:** The names of these two classes were selected after much debate. For `Ket`, we could have used `int` with a fixed endian condition, `str` which we would trust the user to always be composed of `0` and `1`, or a class which could alternatively be named `Ket`, `String`, or `Configuration`. We decided to use `Ket` to force the order, typing, and correctness of this concept, and to make the indexing of `ProbabilityHistogram` and `CountHistogram` also support direct indexing by `str` (cast to `Ket` under the hood) to make things easy. For `ProbabilityHistogram`, we could have used a `dict` or other map, or a class which could alternatively be named `Histogram`, `Counts`, `Shots`, `Probabilities` or any variation thereof. We decided to use `MeasurementResult` to keep things maximally explicit. Note that most users will encounter these objects as the output of `quasar` library methods, and will rarely need to explicitly construct `Ket`, `ProbabilityHistogram`, or `CountHistogram` objects. In this case, the manipulation of these data structures seems straightforward, and allows for easy casting to other data structures that the user might prefer.

## The ProbabilityHistogram and CountHistogram Objects

To standardize the representation of the full histogram of results of a complete measurement process, `quasar` provides the `ProbabilityHistogram` object,

In [2]:
probabilities = quasar.ProbabilityHistogram(
  nqubit=4,
  histogram={
     3  : 0.2, #  3 is 0011
     12 : 0.8, # 12 is 1100
  },
  nmeasurement=1000,
)
print(probabilities)

nqubit       : 4
nmeasurement : 1000
|0011> : 0.200000
|1100> : 0.800000



`ProbabilityHistogram` supports read-only indexing:

In [3]:
print(probabilities[3])

0.2


For convenience, you can also use `str` objects to index a `ProbabilityHistogram`:

In [4]:
print(probabilities['0011'])

0.2


The total number of measurements is provided in the `nmeasurement` attribute:

In [5]:
print(probabilities.nmeasurement)

1000


The number of qubits is provided in the `nqubit` attribute:

In [6]:
print(probabilities.nqubit)

4


Many users are more familiar with the finite integer counts of each `Ket`, as opposed to the probabilities of each `Ket`. To deal with the former, we provide the utility to convert from a `ProbabilityHistogram` (based on floating point probabilities) to a `CountHistogram` (based on integral counts):

In [7]:
counts = probabilities.to_count_histogram()
print(counts)
print(counts['0011'])

nqubit       : 4
nmeasurement : 1000
|0011> : 200
|1100> : 800

200


One can also convert the other direction from a `CountHistogram` to a `ProbabilityHistogram`:

In [8]:
probabilities2 = counts.to_probability_histogram()
print(probabilities2)
print(probabilities2['0011'])

nqubit       : 4
nmeasurement : 1000
|0011> : 0.200000
|1100> : 0.800000

0.2


## Infinite Sampling

In working computations, the `ProbabilityHistogram` is preferred, as it allows for conceptually ideal infinite statistical sampling (indicated by `nmeasurement=None` to indicate infinite `nmeasurement`). No corresponding `CountHistogram` is valid (as all the counts would be infinite), so an error is thrown if one tries to convert an infinitely-sampled `ProbabilityHistogram` to `CountHistogram`:

In [9]:
probabilities = quasar.ProbabilityHistogram(
 nqubit=4,
 histogram={
    3  : 0.2,  #  3 is 0011
    12 : 0.8,  # 12 is 1100
 },
 nmeasurement=None,
)
print(probabilities)
# Throws due to infinite sampling
# counts = probabilities.to_count_histogram()

nqubit       : 4
nmeasurement : None
|0011> : 0.200000
|1100> : 0.800000



## Measurement in Action

Here we show the main place a user would encounter these objects: calling `run_measurement` to sample the output of a given quantum circuit:

In [10]:
circuit = quasar.Circuit().H(0).CX(0,1).CX(1,2)
print(circuit)

T  : |0|1|2|

q0 : -H-@---
        |   
q1 : ---X-@-
          | 
q2 : -----X-
            
T  : |0|1|2|



Here is an example with finite `nmeasurement`, which returns as a `ProbabilityHistogram` and is convertible to a `CountHistogram`.

We'll be using Forge to do the calculations on the server side, so we'll use the Forge client library's "QuasarBackend" class, which
takes a backend string as an argument.  The Forge quasar backend looks a lot like a regular quasar backend, with one notable exception:
all arguments must be "keyword arguments" (there are no positional arguments)

In [11]:
backend = QuasarBackend('qcware/cpu_simulator')
print(backend.summary_str())

Quasar: An Ultralite Quantum Circuit Simulator
   By Rob Parrish (rob.parrish@qcware.com)    


In [12]:
probabilities = backend.run_measurement(circuit=circuit, nmeasurement=1000)
print(probabilities)
counts = probabilities.to_count_histogram()
print(counts)

nqubit       : 3
nmeasurement : 1000
|000> : 0.501000
|111> : 0.499000

nqubit       : 3
nmeasurement : 1000
|000> : 501
|111> : 499



Here is an example with infinite `nmeasurement`, which returns as a `ProbabilityHistogram` and is **not** convertible to a `CountHistogram`:

In [13]:
probabilities = backend.run_measurement(circuit=circuit, nmeasurement=None)
print(probabilities)
# Throws due to infinite sampling
# counts = probabilities.to_count_histogram()
# print(counts)

nqubit       : 3
nmeasurement : None
|000> : 0.500000
|111> : 0.500000

